# System Design

In this notebook we note the operations performed by PVSyst in order to design the system.

It is necessary to indicate the kWp of the system at DC. The number of modules necessary is automatically calculated by PVSyst, given the characteristics of the PV modules selected.

The number of parallel modules is again calculated automatically.

The number of inverters needed is calculated by PVSyst, once again, automatically. The number of inverters is first calculated as

*n_inverters = dc_kwp / ac_inverter_nominal_power*

rounded up.

100kW
30kW

More details are taken into consideration, then. For each inverter, it calculates the maximum number of modules that can be connected in parallel to the inverter (number of strings). To do so, it takes into consideration the electrical limits of the inverter(s):

1. maximum input voltage
2. maximum input current
3. maximum and minimum voltage

An economic analysis is performed, minimizing LCOE, to understand whether it is better to undersize inverters, oversize them or keeping the initial number of inverters estimate.

Another useful KPI is the oversizing ratio:

*oversizing_ratio = dc_kwp / sum(ac_inverter_nominal_power)*

This parameter is often between 0.9 and 1.3. Having a ratio above 1 means enabling clipping. Such choice, though, is often accepted due to inverter's high costs.

In [20]:
# Find one compatible module/inverter pair from your loaded CEC tables
# (uses the same calc you already have under the hood)
import numpy as np
import pvlib

def module_iv_at(mp, Tcell, poa=1000):
    calc = pvlib.pvsystem.calcparams_cec(
        effective_irradiance=poa, temp_cell=Tcell,
        alpha_sc=float(mp['alpha_sc']), a_ref=float(mp['a_ref']),
        I_L_ref=float(mp['I_L_ref']), I_o_ref=float(mp['I_o_ref']),
        R_sh_ref=float(mp['R_sh_ref']), R_s=float(mp['R_s']),
        Adjust=float(mp.get('Adjust', 1.0)),
        EgRef=float(mp.get('EgRef', 1.121)), dEgdT=float(mp.get('dEgdT', -0.0002677)),
    )
    sd = pvlib.pvsystem.singlediode(*calc, method='lambertw')
    return float(sd['v_mp']), float(sd['v_oc'])

def find_first_ok_pair(cecm, ceci, Tcell_hot=65.0, Tcell_cold=0.0, voc_margin=0.97):
    for mname, mp in cecm.items():
        # skip incomplete rows
        need = ['alpha_sc','a_ref','I_L_ref','I_o_ref','R_sh_ref','R_s','V_mp_ref','I_mp_ref']
        if any(k not in mp for k in need): 
            continue
        try:
            Vmp_hot,  Voc_hot_dummy  = module_iv_at(mp, Tcell_hot)
            Vmp_cold, Voc_cold       = module_iv_at(mp, Tcell_cold)
        except Exception:
            continue
        for iname, inv in ceci.items():
            try:
                mppt_low  = float(inv['Mppt_low'])
                mppt_high = float(inv['Mppt_high'])
                vdcmax    = float(inv['Vdcmax'])
            except Exception:
                continue
            n_min = int(np.ceil (mppt_low  / max(Vmp_hot,  1e-6)))
            n_max_mppt = int(np.floor(mppt_high / max(Vmp_cold, 1e-6)))
            n_max_voc  = int(np.floor(voc_margin * vdcmax / max(Voc_cold, 1e-6)))
            n_max = min(n_max_mppt, n_max_voc)
            if n_min <= n_max and n_max >= 1:
                return (mname, iname, n_min, n_max)
    return None


In [27]:
import math, numpy as np, pvlib

def _need(mp, keys):
    missing = [k for k in keys if k not in mp or mp[k] is None or (hasattr(mp[k], 'isna') and mp[k].isna())]
    return missing

def module_iv_at_complete(module_params, temp_cell_C=25.0, poa=1000.0):
    """Return (Vmp, Imp, Pmp, Voc, Isc) at given cell temp using the CEC model.
       Robust to a few missing optional CEC fields."""
    # Required CEC keys (except we’ll default EgRef/dEgdT/Adjust)
    required = ['alpha_sc','a_ref','I_L_ref','I_o_ref','R_sh_ref','R_s']
    miss = _need(module_params, required)
    if miss:
        raise ValueError(f"CEC module parameters missing required keys: {miss}")

    # EgRef  = float(module_params.get('EgRef', 1.121))      # eV, crystalline Si typical
    # dEgdT  = float(module_params.get('dEgdT', -0.0002677)) # eV/K, crystalline Si typical
    # Adjust = float(module_params.get('Adjust', 1.0))

    calc = pvlib.pvsystem.calcparams_desoto(
        effective_irradiance=float(poa),
        temp_cell=float(temp_cell_C),
        alpha_sc=float(module_params['alpha_sc']),
        a_ref=float(module_params['a_ref']),
        I_L_ref=float(module_params['I_L_ref']),
        I_o_ref=float(module_params['I_o_ref']),
        R_sh_ref=float(module_params['R_sh_ref']),
        R_s=float(module_params['R_s']),
        # Adjust=Adjust, EgRef=EgRef, dEgdT=dEgdT
    )
    out = pvlib.pvsystem.singlediode(*calc, method='lambertw')
    return float(out['v_mp']), float(out['i_mp']), float(out['p_mp']), float(out['v_oc']), float(out['i_sc'])

def pick_stringing(
        module, 
        inverter, 
        *,
        t_cell_cold=-5.0, t_cell_hot=65.0, poa_w_per_m2=1000.0,
        mppt_low_key='Mppt_low', mppt_high_key='Mppt_high',
        vdcmax_key='Vdcmax', idcmax_key='Idcmax',
        dc_ac_ratio=1.20, voc_margin=0.97, prefer_mid_window=True):
    """Returns (modules_per_string, strings, details) with robust defaults."""
    # Inverter essentials
    need_inv = ['Paco', mppt_low_key, mppt_high_key, vdcmax_key]
    miss_inv = _need(inverter, need_inv)
    if miss_inv:
        raise ValueError(f"Inverter parameters missing keys: {miss_inv}")

    mppt_low  = float(inverter[mppt_low_key])
    mppt_high = float(inverter[mppt_high_key])
    vdcmax    = float(inverter[vdcmax_key])
    idcmax    = float(inverter.get(idcmax_key, np.inf))
    paco      = float(inverter['Paco'])

    # Module IV at extremes
    Vmp_cold, Imp_cold, Pmp_cold, Voc_cold, Isc_cold = module_iv_at_complete(module, t_cell_cold, poa_w_per_m2)
    Vmp_hot,  Imp_hot,  Pmp_hot,  Voc_hot,  Isc_hot  = module_iv_at_complete(module, t_cell_hot,  poa_w_per_m2)

    # Series constraints
    n_series_max_by_mppt = int(np.floor(mppt_high / max(Vmp_cold, 1e-6)))
    n_series_min_by_mppt = int(np.ceil (mppt_low  / max(Vmp_hot,  1e-6)))
    n_series_max_by_voc  = int(np.floor(voc_margin * vdcmax / max(Voc_cold, 1e-6)))

    n_series_min = max(1, n_series_min_by_mppt)
    n_series_max = min(n_series_max_by_mppt, n_series_max_by_voc)

    if n_series_min > n_series_max:
        raise ValueError(
            f"No valid series count. Computed n_series_min={n_series_min} (MPPT low @ hot) "
            f"> n_series_max={n_series_max} (MPPT high/Vdcmax @ cold)."
        )

    # Pick a series count (try to sit near MPPT mid-window at 25°C)
    if prefer_mid_window:
        Vmp_nom, *_ = module_iv_at(module, 25.0, poa_w_per_m2)
        target = 0.5*(mppt_low + mppt_high)
        candidates = range(n_series_min, n_series_max+1)
        # filter any that violate extremes
        candidates = [n for n in candidates if (n*Vmp_cold <= mppt_high) and (n*Vmp_hot >= mppt_low)]
        if not candidates:
            candidates = [n_series_min]
        n_series = min(candidates, key=lambda n: abs(n*Vmp_nom - target))
    else:
        n_series = n_series_min

    # Parallel strings from DC/AC ratio
    Pmp_stc_module = float(module['V_mp_ref']) * float(module['I_mp_ref'])
    Pmp_string_stc = n_series * Pmp_stc_module
    dc_target = dc_ac_ratio * paco
    n_strings = max(1, int(round(dc_target / max(Pmp_string_stc, 1e-9))))

    # Enforce inverter input current at hot
    total_imp_hot = n_strings * Imp_hot
    if total_imp_hot > idcmax:
        n_strings = max(1, int(np.floor(idcmax / max(Imp_hot, 1e-9))))
        total_imp_hot = n_strings * Imp_hot

    details = dict(
        mppt_low=mppt_low, mppt_high=mppt_high, vdcmax=vdcmax, idcmax=idcmax, paco=paco,
        Vmp_cold=Vmp_cold, Vmp_hot=Vmp_hot, Voc_cold=Voc_cold,
        n_series_min_by_mppt=n_series_min_by_mppt,
        n_series_max_by_mppt=n_series_max_by_mppt,
        n_series_max_by_voc=n_series_max_by_voc,
        chosen_series=n_series,
        Pmp_module_stc=Pmp_stc_module,
        Pmp_string_stc=Pmp_string_stc,
        dc_target_W=dc_target,
        chosen_strings=n_strings,
        total_imp_hot=total_imp_hot
    )
    return n_series, n_strings, details


In [28]:
LAT, LON = 41.9028, 12.4964
ALTITUDE_METERS = 65
TRACKING_TYPE = "fixed"  # "fixed" | "single_axis" | "two_axis"
RACKING_MODEL = "open_rack"  # "open_rack" or "close_mount" (used for temp/IAM conventions)
FIXED_TILT = 0
FIXED_AZIMUTH = 0
AXIS_TILT = 0.0
AXIS_AZIMUTH = 180.0       # N-S axis: 0 or 180 (pvlib uses degrees from North, CW)
MAX_ANGLE = 60.0
BACKTRACK = False
GCR = 0.35
K = 0.05
U_C, U_V = (29.0, 0.0) if RACKING_MODEL == "open_rack" else (20.0, 6.0)

cecm = pvlib.pvsystem.retrieve_sam('cecmod')
ceci = pvlib.pvsystem.retrieve_sam('cecinverter')
ok = find_first_ok_pair(cecm, ceci, Tcell_hot=65.0, Tcell_cold=0.0, voc_margin=0.97)
module = cecm[ok[0]]
module['K'] = module.get('K', K)
inverter = ceci[ok[1]]
eta_m = float(module['I_mp_ref']*module['V_mp_ref']) / (1000.0 * float(module['A_c']))

temperature_model_parameters = dict(u_c=U_C, u_v=U_V, eta_m=eta_m, alpha_absorption=0.9)
loc = pvlib.location.Location(LAT, LON, tz="Europe/Rome", altitude=ALTITUDE_METERS)

In [ ]:
# Decide stringing for your selected CEC module & inverter
n_series, n_strings, sizing = pick_stringing(module, inverter, dc_ac_ratio=1.20)

print("Picked:", n_series, "modules per string,", n_strings, "strings")
print("Sanity:", sizing)

Picked: 8 modules per string, 6 strings
Sanity: {'mppt_low': 220.0, 'mppt_high': 416.0, 'vdcmax': 416.0, 'idcmax': 32.775958, 'paco': 10000.0, 'Vmp_cold': 42.288683130550226, 'Vmp_hot': 29.201257141130796, 'Voc_cold': 49.49528305748731, 'n_series_min_by_mppt': 8, 'n_series_max_by_mppt': 9, 'n_series_max_by_voc': 8, 'chosen_series': 8, 'Pmp_module_stc': 175.09140000000002, 'Pmp_string_stc': 1400.7312000000002, 'dc_target_W': 12000.0, 'chosen_strings': 6, 'total_imp_hot': 28.648900769253814}


In [30]:
ok

('A10Green_Technology_A10J_S72_175',
 'ABB__PVI_10_0_I_OUTD_x_US_208_y__208V_',
 8,
 8)